# Quick start and cylindrical cell example

First: Download the cellmodel.jl file and run it. This notebook is tested with Julia v 1.5.3.

**Note**: you must have the packages Measurements, Parameters, QuadGK, DataFrames installed.
Missing packages can be installed by running the following line at the REPL for each package:

`Pkg.add("packagename")`

In [1]:
# Skip this line if it's already in the working folder.
download("https://raw.githubusercontent.com/mjlacey/cellmodels/master/cellmodel.jl", "cellmodel.jl")

# Include the cellmodel.jl file it to load the required packages and functions.
include("cellmodel.jl")

volumetric_energy (generic function with 3 methods)

It is useful to define all materials (active materials, current collectors, separators, electrolytes) bearing in mind that all cell models are constructed in the following manner. Each component starting with a capital letter below is a `struct` in Julia with different properties (some of which are other `structs`), ordered like so:

```
Cell
    - Electrode (+)
        - ElectrodeComposite (+)
            - ActiveMaterial (+)
        - CurrentCollector (+)
    - Electrode
        - ElectrodeComposite (-)
            - ActiveMaterial (-)
        - CurrentCollector (-)
    - Separator
    - Electrolyte
```

There are several different cell types at present.

## 1. Material definitions

### 1.1 Active material definition

Active (e.g. electrode) material definitions are simply made in the form:

`ActiveMaterial("Name", specific_capacity, average_potential)`

Specific capacities have units of mAh/g, average potential in V.

The model does not take voltage cutoffs into account at present so care must be taken to ensure given values are appropriate. Errors (treated as 95% confidence intervals) can be given using the ± operator). Mixtures of materials can be estimated by linear combination.

In [2]:
# Active materials

# Positive electrodes
NMC811 = ActiveMaterial("NMC811", 195 ± 5, 3.86)    # NMC811 determined from literature
NMC622 = ActiveMaterial("NMC622", 181 ± 4.5, 3.86)  # NMC622 determined from literature
NCA = ActiveMaterial("NCA", 210 ± 10, 3.86)         # NCA values estimated

# Negative electrodes
Gr = ActiveMaterial("Graphite", 344 ± 8.6, 0.17)    # Gr determined from literature
GrSi3_5 = ActiveMaterial("3.5% Si/Gr", 
    ((344 ± 8.6) * 0.965) + ((2200 ± 200) * 0.035),
    (0.17 * 0.965) + (0.4 * 0.035))                 # 3.5% Si/Gr estimated from linear combination, assuming
                                                    # Si gives 2200 ± 200 mAh/g @ 0.4 V.

ActiveMaterial
  name: String "3.5% Si/Gr"
  spec_cap: Measurement{Float64}
  avg_E: Float64 0.17805


### 1.2 Current collector definition

Current collectors can be defined using the form:

`CurrentCollector("Element", thickness)`

It recognises Al and Cu, and selects the density accordingly. Thicknesses are given in cm.

In [3]:
# Current collectors
Al14 = CurrentCollector("Al", 14E-4) # 14 µm Al
Cu8 = CurrentCollector("Cu", 8E-4)   # 8 µm Al

CurrentCollector
  name: String "Cu"
  thickness: Float64 0.0008
  density: Float64 8.96
  mass: #undef


### 1.3 Electrolyte definition

Electrolytes are defined in the following manner:

`Electrolyte(salt, solvent, concentration, density, saltmassfrac)`

This is done in a way that for some electrolytes, a concentration can be given from which a density is calculated, according to a known concentration-density relationship. At the moment, this is only available for LiPF6 in EC:DEC from literature values. Otherwise, density should be given, but concentration and saltmassfrac (salt mass fraction) can be left blank.

In [4]:
# Electrolytes
LP40 = Electrolyte(salt = "LiPF6", solvent = "EC:DEC", concentration = 1.1 ± 0.1)

Electrolyte
  salt: String "LiPF6"
  solvent: String "EC:DEC"
  concentration: Measurement{Float64}
  density: Measurement{Float64}
  saltmassfrac: Measurement{Float64}


### 1.4 Separator definition

Separators are defined in the following manner:

`Separator(name, thickness, porosity, density)`.

This is a bit broken at the moment, because the `porosity` argument does nothing - the `density` is the effective density of the separator, not the bulk (100% dense) density.

In [5]:
# Separator
PE_Al12 = Separator(name = "porous PE + alumina", thickness = 12E-4, porosity = 0.44, density = 1.18)

Separator
  name: String "porous PE + alumina"
  thickness: Float64 0.0012
  porosity: Float64 0.44
  density: Float64 1.18


## 2. Cell definition

This example will use the `CylindricalCell` model, initialised using the form `CylindricalCell(params...)`. There are several parameters for a cell. In the below example we will create a basic NMC622/Gr cell using the above-described materials.

Creating a cell model will generate a summary of the key cell properties as output.

In [6]:
example_cylindrical = CylindricalCell(

    name = "Example cylindrical cell",   # A name for the cell

    positive = Electrode(
        ElectrodeComposite(             # An electrode is composed of an electrode composite, which has:
            active_material = NMC622,   # Active material, defined above
            areal_cap = 3.3 ± 0.1,      # An areal capacity, in mAh/cm2
            active_frac = 0.95 ± 0.02,  # An active material fraction, in %
            density = 3.4 ± 0.1         # A (calendered) density, in g/cm3
        ),
        Al14                            # The composite is coated on the current collector, defined above.
    ),
    
    negative = Electrode(               # The negative electrode is similarly defined.
        ElectrodeComposite(
            active_material = Gr,
            areal_cap = (3.3 ± 0.1) * (1.1 ± 0.03),  # Can, if desired, specify n/p ratio in this way.
            active_frac = 0.965 ± 0.01,
            density = 1.6 ± 0.1
        ),
        Cu8
    ),
            
    separator = PE_Al12,                # Separator and electrolyte, as defined above.
    electrolyte = LP40,
    
    # Then, the cell has the following properties and dimensions:
    
    ecap_ratio = 1.7 ± 0.1,              # Electrolyte/capacity ratio, in mL/Ah; used for calculating electrolyte mass
    
    diameter = 2.1, height = 7.0,       # For a 2170 cell
    canthickness = 0.0165,              # Thickness of the cell can, in cm
    candensity = 7.9 ± 0.2,             # Density of the can material, in this case steel
    voiddiameter = 0.25,                # The diameter of the 'void'/empty space in the centre of the winding/jelly roll, in cm
    headspace = 0.6,                    # The distance from the top of the can to the top of the jelly roll, in cm
    extramass = 4 ± 2,                  # A fudge factor; any other unaccounted mass (terminals, safety features), in g
    
    llifactor = 0.94 ± 0.02             # Another fudge factor; adjustment of ideal capacity accounting for first cycle loss.
                                        # 6% first cycle loss (typical) would be 0.94 for this parameter.
)

Example cylindrical cell
NMC622 cathode @ 3.3 ± 0.1 mAh/cm2
4.29 ± 0.22 Ah, 15.84 ± 0.79 Wh
71.1 ± 2.9 g
222.8 ± 9.4 Wh/kg
653.0 ± 33.0 Wh/L


## 3. Examples of cell property analysis

Properties of each cell, as defined by

`cell = CylindricalCell(params...)`

can be accessed e.g. by

`cell.property`

Creating a cell model creates three new properties, `jr_area` (jelly roll area, in cm2), `capacity` (capacity of the cell, in Ah) and `energy` (energy of the cell, in Wh).

There are several functions available currently for analysing cell properties, which work with many (but perhaps not all) cell types, or cell properties. Examples of functions include:

`thickness()`

`mass()`

`massbreakdown()`

`gravimetric_energy()`

`volumetric_energy()`

Some examples follow below.

In [7]:
# Thickness of the positive electrode (current collector + double sided coating, in cm)
thickness(example_cylindrical.positive)

0.01269 ± 0.0006

In [8]:
# Thickness of the positive electrode *coating* only, in cm
thickness(example_cylindrical.positive.composite)

0.00564 ± 0.0003

In [9]:
# Mass of the separator, in mg/cm2. To get cell-level information, multiply by jr_area
mass(example_cylindrical.separator)

0.0014159999999999997

In [10]:
# Total mass of separator in the cell
mass(example_cylindrical.separator) * example_cylindrical.jr_area

0.98 ± 0.042

In [11]:
# Gravimetric energy density of the cell, in Wh/kg
gravimetric_energy(example_cylindrical)

222.8 ± 9.4

In [12]:
# Mass breakdown of the cell. Masses in g.
massbreakdown(example_cylindrical)

,component,mass,percentage
,String,Number,Measure…
1,+ve cc,2.61±0.11,3.68±0.15
2,+ve composite,26.6±1.3,37.4±1.5
3,-ve cc,4.96±0.21,6.98±0.28
4,-ve composite,15.13±0.67,21.3±1.1
5,separator,1.959±0.085,2.76±0.11
6,electrolyte,8.99±0.7,12.65±0.84
7,packaging,10.9±2.0,15.3±2.4
